In [ ]:
import torch
import os
from huggingface_hub import HfApi
from pathlib import Path
from diffusers.utils import load_image
from PIL import Image
import numpy as np
from controlnet_aux import PidiNetDetector, HEDdetector

from diffusers import (
    ControlNetModel,
    StableDiffusionControlNetPipeline,
    UniPCMultistepScheduler,
)

import gradio as gr
import cv2
import numpy as np
from PIL import Image
from gradio_client import Client
import os
import uuid
import random

checkpoint = "lllyasviel/control_v11p_sd15_scribble"

processor = HEDdetector.from_pretrained('lllyasviel/Annotators')

controlnet = ControlNetModel.from_pretrained(checkpoint, torch_dtype=torch.float16)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()


In [ ]:
def create_directory(directory_name='images'):
  try:
    os.mkdir(directory_name)
    print(f"Directory '{directory_name}' created successfully.")
  except FileExistsError:
    print(f"Directory '{directory_name}' already exists.")
  except PermissionError:
    print(f"Permission denied: Unable to create '{directory_name}'.")
  except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
def make_collage(path_img1, path_img2, uuid):
    # Carregar as duas imagens
    imagem1 = Image.open(path_img1)  # Imagem 1920x1440
    imagem2 = Image.open(path_img2)  # Imagem 704x512

    # Redimensionar a primeira imagem para 704px de largura, mantendo o aspect ratio
    nova_largura = 704
    proporcao = nova_largura / imagem1.width
    nova_altura = int(imagem1.height * proporcao)
    imagem1 = imagem1.resize((nova_largura, nova_altura))

    # Calcular a altura total para a colagem (precisamos de 1024 de altura)
    altura_total = 1024

    # Criar uma nova imagem com 704x1024 e fundo branco
    imagem_final = Image.new("RGB", (nova_largura, altura_total), "white")

    # Colar a primeira imagem no topo da imagem final
    imagem_final.paste(imagem1, (0, 0))

    # Colar a segunda imagem logo abaixo da primeira
    imagem_final.paste(imagem2, (0, imagem1.height))

    # Salvar o resultado
    imagem_final.save(f"images/ARENA IA - CIn - Colagens/{uuid}_collage.jpg")

In [ ]:
def create_image(image_path, prompt, n_prompt):
  image = load_image(image_path)
  print(type(image))
  
  control_image = processor(image, scribble=True)

  seed_input = random.randint(0, 1000000)
  generator = torch.manual_seed(seed_input)
 
  # n_prompt = "longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, hands, flesh, skin, human, glass, sky, plant, faceless, paint, pillow, complex background" # "complex background, faceless"
 
  image2 = pipe(prompt, num_inference_steps=22, generator=generator, image=control_image, negative_prompt=n_prompt, guidance_scale=12).images[0]
 
  image_uuid = str(uuid.uuid4())

  path_o_img = 'images/' + image_uuid + '_original.png'
  path_g_img = 'images/' + image_uuid + '_generated.png'

  image.save(path_o_img)
  image2.save(path_g_img)
  make_collage(path_o_img, path_g_img, image_uuid)
 
  return image2

In [77]:
def process_image(image, prompt_option):
    match prompt_option:
        case "Pelúcia":
            return create_image(image, 
                                prompt="A picture of a stuffed animal based on image provided with best quality, extremely detailed, pulsh, felt, face, lifelike",
                                n_prompt="longbody, lowers, bad, anatomy, bad hands, fewer digits, cropped, worst quality, hands, flesh, skin, human, glass, sky, plant, faceless, paint, pillow")
        case "Objeto":
            return create_image(image, 
                                prompt="An object based on image provided with best quality, extremely detailed, object, simple background, white blackground, plain background, tools, utensils", 
                                n_prompt="longbody, lowers, bad anatomy, bad hands, missing fingers, extra digits, cropped, worst quality, low quality, complex background, plant, food, natural things")
        case "Paisagem":
            return create_image(image, 
                                prompt="A scenic view based on image provided with best quality extremely detailed, landscape, nature, trees, rivers, moutains, grass, leaves, wood, trunk, flowers, plants, natural, beautiful, sun, sunny, interpret circle in the sky as the sun, lively, sky clouds, wooden, cottage, creek, waterfall, warm, beach, sand, sea, ocean, palm trees, scenary", 
                                n_prompt="longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, industrial, manmade, metal, machine, ugly")
        case "Cartoon":
            return create_image(image, 
                                prompt="An image of a cartoon character based on image provided with best quality, extremely detailed, simple background, character, face, hair, clothes, good hands, cartoon, sketch, drawing, art, interpret stick figures as humans", 
                                n_prompt="longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, complex background, faceless, uncanny, scary, real human")
        case _:
            return create_image(image, 
                                prompt="A picture of a stuffed animal based on image provided with best quality, extremely detailed, pulsh, felt, face, lifelike",
                                n_prompt="longbody, lowers, bad, anatomy, bad hands, fewer digits, cropped, worst quality, hands, flesh, skin, human, glass, sky, plant, faceless, paint, pillow")
        

In [ ]:
create_directory()

# Rodar o Gradio

In [ ]:
image_input = gr.Image(type="filepath", label="Enviar imagem", mirror_webcam=False, sources=["webcam"])
prompts = gr.Radio(
    ["Pelúcia", "Objeto", "Paisagem", "Cartoon"],
    label="Selecione uma opção"
)

iface = gr.Interface(
    fn=process_image,
    # fn=create_image,
    inputs=[image_input, prompts],
    # inputs=image_input,
    outputs=gr.Image(label="Imagem processada"),
    live=False,
    allow_flagging='never',
    title="",
    css="""
        @import url('https://fonts.googleapis.com/css2?family=Nunito:wght@400;700&display=swap');

        .gradio-container {
            font-family: 'Nunito', sans-serif;
            background: linear-gradient(#000000 25%, #ff0571 75%, #f98404); # verde 1eff05, laranja f98404, roxo 52057b, rosa ff0571
            color: #ffffff;
            text-align: center;
        }
        h1 {
            width: 100%;
            margin-top: -30px;
            max-width: 300px;
            display: block;
            margin-left: auto;
            margin-right: auto;
        }
    """
)

# Inicia a interface
iface.launch(debug=True)

Directory 'images' already exists.


c:\Users\FCAC\git\demo-criacom\venv\Lib\site-packages\gradio\interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


<class 'PIL.Image.Image'>


100%|██████████| 22/22 [00:07<00:00,  2.85it/s]


...

<class 'PIL.Image.Image'>


100%|██████████| 22/22 [00:06<00:00,  3.47it/s]


<class 'PIL.Image.Image'>


100%|██████████| 22/22 [00:06<00:00,  3.51it/s]


Keyboard interruption in main thread... closing server.


In [ ]:
# This is a test message